In [2]:
#!pip install bert-for-tf2
#!pip install sentencepiece

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

from numpy import loadtxt

import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [4]:
import numpy as np
import pandas as pd
import string

import transformers as ppb
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten

In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re as re

In [6]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [7]:
train_data['keyword'].fillna('no keyword', inplace = True) 
train_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
train_data['location'].fillna('no location', inplace = True)
train_data.drop(['id'],1,inplace = True)

In [8]:
test_data['keyword'].fillna('no keyword', inplace = True)
test_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
test_data['location'].fillna('no location', inplace = True)

In [9]:
#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

custom_sent_tokenizer = PunktSentenceTokenizer()

def process_content(sentence):
    tokenized = custom_sent_tokenizer.tokenize(sentence)
    words_tagged = []
    for i in tokenized:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        for word in tagged:
            words_tagged.append(word)
            
    return words_tagged

#Cleaning text

train_data['clean_text'] = train_data['text'].str.lower()
test_data['clean_text'] = test_data['text'].str.lower()

train_data['clean_text'] = train_data['clean_text'].apply(only_letters)
test_data['clean_text'] = test_data['clean_text'].apply(only_letters)   

#Tokenización

train_data['clean_text'] = train_data['clean_text'].apply(word_tokenize)
test_data['clean_text'] = test_data['clean_text'].apply(word_tokenize)

#Remove stopwords

train_data['clean_text'] = train_data['clean_text'].apply(filter_stopwords)
test_data['clean_text'] = test_data['clean_text'].apply(filter_stopwords)

#Lemmatization                                                                       

train_data['clean_text'] = train_data['clean_text'].apply(lemmatize_tweet)
test_data['clean_text'] = test_data['clean_text'].apply(lemmatize_tweet)

#Transform to text
train_data['clean_text'] = train_data['clean_text'].apply(transform_to_text)
test_data['clean_text'] = test_data['clean_text'].apply(transform_to_text)

test_data
#Part of speech tagging

train_data['tagged_text']= train_data['text'].apply(process_content)
train_data['tagged_clean_text']= train_data['clean_text'].apply(process_content)

test_data['tagged_text']= test_data['text'].apply(process_content)
test_data['tagged_clean_text']= test_data['clean_text'].apply(process_content)

In [10]:
def count_Nouns(list):
    nouns = 0
    for word,tag in list:
        if tag[0] == 'N':
            nouns += 1
        else:
            continue
    return nouns

def count_Adjetives(list):
    adjetives = 0
    for word,tag in list:
        if tag[0] == 'J':
            adjetives += 1
        else:
            continue
    return adjetives

def count_Verbs(list):
    verbs = 0
    for word,tag in list:
        if tag[0] == 'V':
            verbs += 1
        else:
            continue
    return verbs 


# word_count
train_data['word_count'] = train_data['text'].apply(lambda x: len(str(x).split()))
test_data['word_count'] = test_data['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
train_data['unique_word_count'] = train_data['text'].apply(lambda x: len(set(str(x).split())))
test_data['unique_word_count'] = test_data['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
train_data['stop_word_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
test_data['stop_word_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# url_count
train_data['url_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data['url_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
train_data['mean_word_length'] = train_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data['mean_word_length'] = test_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# length
train_data['tweet_length'] = train_data['text'].apply(lambda x: len(str(x)))
test_data['tweet_length'] = test_data['text'].apply(lambda x: len(str(x)))

# punctuation_count
train_data['punctuation_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_data['punctuation_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
train_data['hashtag_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test_data['hashtag_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
train_data['mention_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test_data['mention_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

#noun_count
train_data['noun_count'] = train_data['tagged_text'].apply(count_Nouns)
test_data['noun_count'] = test_data['tagged_text'].apply(count_Nouns)


#verb_count
train_data['verb_count'] = train_data['tagged_text'].apply(count_Verbs)
test_data['verb_count'] = test_data['tagged_text'].apply(count_Verbs)

#adjetives_count
train_data['adjetives_count'] = train_data['tagged_text'].apply(count_Adjetives)
test_data['adjetives_count'] = test_data['tagged_text'].apply(count_Adjetives)


### Cargamos el modelo: Pre-trained BERT 


In [11]:
#DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#BERT: 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Cargo pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

100%|██████████| 440473133/440473133 [02:19<00:00, 3159652.46B/s]


In [12]:
#Usamos el tokenizador de BERT para dividir los tweets en tokens

tokenized = train_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#tokenized = train_data['clean_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

tokenized_test = test_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#tokenized_test = test_data['clean_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [13]:
#Necesitamos que todos los vectores tengan el mismo tamaño por lo que les ponemos 
#un padding para que BERT pueda interpretarlos

#TRAIN
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

#TEST
max_len_test = 0
for i in tokenized_test.values:
    if len(i) > max_len_test:
        max_len_test = len(i)

padded_test = np.array([i + [0]*(max_len_test-len(i)) for i in tokenized_test.values])

In [14]:
np.array(padded).shape

(7613, 84)

In [15]:
np.array(padded_test).shape

(3263, 73)

In [16]:
#Si le mandamos directamente los vectores con el padding realizado BERT puede confundirse
# Necesitamos pasarle otra variable que le diga a nuestro modelo el padding que agregamos cuando procesa el input.

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(7613, 84)

In [17]:
attention_mask_test = np.where(padded_test != 0, 1, 0) 
attention_mask_test.shape

(3263, 73)

In [19]:
#Creamos el tensor para pasarle al modelo de bert
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
#last_hidden_states es una tupla con la forma: numero de rows, mayor numero de tokens en una secuencia, 
# numero de "hidden units" en el modelo de BERT (en nuestro caso 768)


In [ ]:
input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states_test = model(input_ids_test, attention_mask=attention_mask_test)

In [52]:
#Para clasificar los tweets solo nos interesa del output el [CLS] Token, por lo que seleccionamos la primera posición
#de todos los tweets y nos quedamos tambien con los "hidden unit outputs"

features = last_hidden_states[0][:,0,:].numpy()
features_test = last_hidden_states_test[0][:,0,:].numpy()

# Ahora features es un array de 2 dimensiones que contienen los embeddings de todas los tweets del data set.


In [53]:
len(features)


7613

In [ ]:
len(features_test)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(features, train_data['target'].values, test_size=0.20, random_state=42)

## Red neuronal

In [43]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# define the keras model
model = keras.Sequential()
model.add(keras.layers.Dense(12, input_dim=768, activation='relu'))
#model.add(Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=12, batch_size=10)

Train on 6090 samples
Epoch 1/12
6090/6090 [==============================] - 2s 301us/sample - loss: 0.4697 - accuracy: 0.7780 - f1_m: 0.6828 - precision_m: 0.7808 - recall_m: 0.6615
Epoch 2/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4232 - accuracy: 0.8087 - f1_m: 0.7247 - precision_m: 0.8342 - recall_m: 0.6846
Epoch 3/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4169 - accuracy: 0.8108 - f1_m: 0.7304 - precision_m: 0.8248 - recall_m: 0.7007
Epoch 4/12
6090/6090 [==============================] - 1s 191us/sample - loss: 0.4105 - accuracy: 0.8174 - f1_m: 0.7359 - precision_m: 0.8354 - recall_m: 0.6984
Epoch 5/12
6090/6090 [==============================] - 1s 192us/sample - loss: 0.4034 - accuracy: 0.8215 - f1_m: 0.7466 - precision_m: 0.8449 - recall_m: 0.7141
Epoch 6/12
6090/6090 [==============================] - 1s 219us/sample - loss: 0.3983 - accuracy: 0.8222 - f1_m: 0.7438 - precision_m: 0.8483 - recall_m: 0.7058
Epoch 

In [44]:
# evaluate the keras model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
print('f1: %.2f' % (f1_score*100))

1523/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Accuracy: 82.80
f1: 78.02


In [ ]:
predictions = model.predict_classes(features_test)

In [ ]:
test_data['target'] = predictions.astype(int)
submission = test_data.loc[:,['id','target']]
len(submission)
submission.to_csv(path_or_buf='submissionDEEPLEARNING+BERT.csv',header=True,index=False)

In [ ]:
#Distribución de las predicciones 

submission['target'].value_counts()

## XGBOOST

In [ ]:
import xgboost as xgb


clf = xgb.XGBClassifier(max_depth=200, n_estimators=400, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)

clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1score = f1_score(y_test, predictions)
print(f'Counts model score: {f1score*100}%')

In [ ]:
predictions = clf.predict(features_test)

In [ ]:
test_data['target'] = predictions.astype(int)
submission = test_data.loc[:,['id','target']]
len(submission)
submission.to_csv(path_or_buf='submissionXGB+BERT.csv',header=True,index=False)

In [ ]:
#Distribución de las predicciones 
submission['target'].value_counts()

## Ensamble de modelos 

In [ ]:
preds_xgb = pd.read_csv('submissionXGB+BERT.csv')['target']
preds_deep = pd.read_csv('submissionDEEPLEARNINGBERT.csv')['target']

In [ ]:
avg = (preds_deep + preds_xgb)/2
avg = (avg >= 0.5).astype(int)

test_data['target'] = avg
submission = test_data.loc[:,['id','target']]
submission.to_csv(path_or_buf='submissionavgdeep-xgb.csv',header=True,index=False)


In [ ]:
#Distribución de las predicciones 

submission['target'].value_counts()